In [ ]:
import os
import numpy as np
import open3d
import time
from utils.metric import ConfusionMatrix
from utils.point_cloud_util import load_labels, colorize_point_cloud
import multiprocessing

In [ ]:
sparse_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/sparse"
dense_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/dense"
gt_dir = "/home/ylao/data/semantic3d"

In [ ]:
file_prefix = "untermaederbrunnen_station1_xyz_intensity_rgb"

# Sparse labels
sparse_labels = np.array(load_labels(os.path.join(sparse_dir, file_prefix + ".labels")), dtype=np.int32)

# Sparse points
sparse_pcd = open3d.read_point_cloud(os.path.join(sparse_dir, file_prefix + ".pcd"))
sparse_points = np.asarray(sparse_pcd.points)

# Dense points
dense_pcd = open3d.read_point_cloud(os.path.join(gt_dir, file_prefix + ".pcd"))
dense_points = np.asarray(dense_pcd.points)

# Ground-trugh dense labels
gt_labels = load_labels(os.path.join(gt_dir, file_prefix + ".labels"))

In [ ]:
# Build KNN tree
sparse_pcd_tree = open3d.KDTreeFlann(sparse_pcd)

In [ ]:
def match_knn_label(dense_index):
    global dense_points
    global sparse_labels
    global sparse_pcd_tree
    
    dense_point = dense_points[dense_index]
    k, sparse_indexes, _ = sparse_pcd_tree.search_knn_vector_3d(dense_point, 10)
    knn_sparse_labels = sparse_labels[sparse_indexes]
    dense_label = np.bincount(knn_sparse_labels).argmax()
    return dense_label

# Assign labels
start = time.time()
dense_indexes = list(range(len(dense_points)))
with multiprocessing.Pool() as pool:
    dense_labels = pool.map(match_knn_label, dense_indexes)
print("knn match time: ", time.time() - start)

# Eval
cm = ConfusionMatrix(9)
cm.increment_from_list(gt_labels, dense_labels)
cm.print_metrics()